In [1]:
import os
import sys
import asyncio
import time
import logging
from pathlib import Path
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Check environment
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found in environment")
    print("   Run: source ../.env && export OPENAI_API_KEY")
else:
    print("✅ OPENAI_API_KEY found in environment")

# Set up clean logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Keep important logs but reduce noise
logging.getLogger('autogen_core.events').setLevel(logging.WARNING)
logging.getLogger('autogen_core').setLevel(logging.WARNING)
logging.getLogger('autogen_agentchat').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpcore').setLevel(logging.WARNING)

print("✅ Environment and logging configured")

✅ OPENAI_API_KEY found in environment
✅ Environment and logging configured


In [2]:
# Import all required components
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager
from schema_reader import SchemaReader

# All agents + task status checker
from query_analyzer_agent import QueryAnalyzerAgent
from schema_linker_agent import SchemaLinkerAgent
from sql_generator_agent import SQLGeneratorAgent
from sql_evaluator_agent import SQLEvaluatorAgent
from task_status_checker import TaskStatusChecker, TaskStatusCheckerArgs

# Memory types
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    TableSchema, ColumnInfo, ExecutionResult
)

## 1. Initialize Shared Memory System

Set up the shared memory system and all managers that coordinate the workflow.

In [3]:
# Initialize shared memory system - this is the foundation for all agents
print("🧠 Initializing shared memory system...")
memory = KeyValueMemory()

# Initialize all managers that coordinate workflow
print("📋 Initializing workflow managers...")
task_manager = TaskContextManager(memory)
tree_manager = QueryTreeManager(memory)
schema_manager = DatabaseSchemaManager(memory)
history_manager = NodeHistoryManager(memory)

print("✅ Shared memory and managers initialized")
print("   - Memory system ready for multi-agent coordination")
print("   - All managers connected to shared memory")

🧠 Initializing shared memory system...
📋 Initializing workflow managers...
✅ Shared memory and managers initialized
   - Memory system ready for multi-agent coordination
   - All managers connected to shared memory


## 2. Initialize Task Context and Load Database Schema

Set up the task context and load database schema into shared memory.

In [4]:
# Database configuration
data_path = "/home/norman/work/text-to-sql/MAC-SQL/data/bird"
tables_json_path = Path(data_path) / "dev_tables.json"
db_name = "california_schools"

# Test query
test_query = "What is the highest eligible free rate for K-12 students in the schools in Alameda County?"
evidence = "Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`"
task_id = "workflow_demo_001"

print(f"🎯 Target Query: {test_query}")
print(f"🎯 Evidence: {evidence}")
print(f"📊 Database: {db_name}")
print("─" * 80)

# Initialize task context in shared memory
print("📝 Initializing task context in shared memory...")
await task_manager.initialize(task_id, test_query, db_name, evidence)

# Load schema using SchemaReader
print("📚 Loading database schema into shared memory...")
schema_reader = SchemaReader(
    data_path=data_path,
    tables_json_path=str(tables_json_path),
    dataset_name="bird",
    lazy=False
)

# Load schema into shared memory
await schema_manager.load_from_schema_reader(schema_reader, db_name)

# Verify memory state
print("✅ Task context and schema loaded into shared memory")
print("   - Task context available to all agents")
print("   - Database schema accessible by all agents")
print("   - Shared memory ready for multi-agent workflow")

# Initialize the query tree with root node before starting workflow
print("🌳 Initializing query tree with root node...")
root_id = await tree_manager.initialize(test_query, evidence)
print(f"✅ Query tree initialized with root node: {root_id}")
print("   - Root node contains the original query")
print("   - Current node set to root")
print("   - Ready for multi-agent workflow")

2025-05-30 03:42:09,070 - TaskContextManager - INFO - Initialized task context for task workflow_demo_001
2025-05-30 03:42:09,070 - TaskContextManager - INFO - Evidence provided: Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`


🎯 Target Query: What is the highest eligible free rate for K-12 students in the schools in Alameda County?
🎯 Evidence: Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`
📊 Database: california_schools
────────────────────────────────────────────────────────────────────────────────
📝 Initializing task context in shared memory...
📚 Loading database schema into shared memory...
load json file from /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_tables.json

Loading all database info...
Found 11 databases in bird dataset


2025-05-30 03:42:21,583 - DatabaseSchemaManager - INFO - Initialized empty database schema
2025-05-30 03:42:21,584 - DatabaseSchemaManager - INFO - Added table 'frpm' to schema
2025-05-30 03:42:21,584 - DatabaseSchemaManager - INFO - Added table 'satscores' to schema
2025-05-30 03:42:21,585 - DatabaseSchemaManager - INFO - Added table 'schools' to schema
2025-05-30 03:42:21,585 - DatabaseSchemaManager - INFO - Loaded schema for database 'california_schools' with 3 tables
2025-05-30 03:42:21,586 - QueryTreeManager - INFO - Initialized query tree with root node root
2025-05-30 03:42:21,586 - QueryTreeManager - INFO - Root node includes evidence: Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`


✅ Task context and schema loaded into shared memory
   - Task context available to all agents
   - Database schema accessible by all agents
   - Shared memory ready for multi-agent workflow
🌳 Initializing query tree with root node...
✅ Query tree initialized with root node: root
   - Root node contains the original query
   - Current node set to root
   - Ready for multi-agent workflow


In [5]:
snapshot = await memory.show_all('table')
print(snapshot)

                             MEMORY STORE CONTENTS                              
Key                            Type            Size       Value Preview            
--------------------------------------------------------------------------------
queryTree                      JSON            373 chars  Dict with 3 keys         
databaseSchema                 JSON            16946 chars Dict with 2 keys         
taskContext                    JSON            341 chars  Dict with 6 keys         
Total unique keys: 3
Total stored items: 6


## 3. Initialize All Agents with Shared Memory

Create instances of all specialized agents, each connected to the shared memory system.

In [6]:
# LLM configuration for all agents
llm_config = {
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}

print("🤖 Initializing all agents with shared memory connection...")

# Initialize all agents with shared memory access
query_analyzer = QueryAnalyzerAgent(memory, llm_config)
schema_linker = SchemaLinkerAgent(memory, llm_config)
sql_generator = SQLGeneratorAgent(memory, llm_config)
sql_evaluator = SQLEvaluatorAgent(memory, llm_config)

# Initialize TaskStatusChecker (deterministic - no LLM needed)
task_status_checker = TaskStatusChecker(memory)

print("✅ All agents initialized and connected to shared memory")
print("   - QueryAnalyzerAgent: Ready for query decomposition")
print("   - SchemaLinkerAgent: Ready for schema mapping")
print("   - SQLGeneratorAgent: Ready for SQL generation")
print("   - SQLEvaluatorAgent: Ready for execution and evaluation")
print("   - TaskStatusChecker: Ready for workflow coordination")
print("   - All agents share the same memory space for coordination")

2025-05-30 03:42:21,614 - QueryAnalyzerAgent - INFO - Initialized query_analyzer with model gpt-4o
2025-05-30 03:42:21,625 - SchemaLinkerAgent - INFO - Initialized schema_linker with model gpt-4o
2025-05-30 03:42:21,636 - SQLGeneratorAgent - INFO - Initialized sql_generator with model gpt-4o
2025-05-30 03:42:21,647 - SQLEvaluatorAgent - INFO - Initialized sql_evaluator with model gpt-4o


🤖 Initializing all agents with shared memory connection...
✅ All agents initialized and connected to shared memory
   - QueryAnalyzerAgent: Ready for query decomposition
   - SchemaLinkerAgent: Ready for schema mapping
   - SQLGeneratorAgent: Ready for SQL generation
   - SQLEvaluatorAgent: Ready for execution and evaluation
   - TaskStatusChecker: Ready for workflow coordination
   - All agents share the same memory space for coordination


## 4. Manual Workflow Execution

Execute the multi-agent workflow step by step, with each agent operating on the shared memory.

In [7]:
# Step 1: Schema Linking
print("\n🔗 STEP 1: SCHEMA LINKING")
print("=" * 50)

# Get current node
current_node_id = await tree_manager.get_current_node_id()
current_node = await tree_manager.get_node(current_node_id)
print(f"Working on node: {current_node_id}")
print(f"Intent: {current_node.intent}")

# Run schema linker
result = await schema_linker.run("Link query to database schema")
print(f"✅ Schema linking complete")

# Check the mapping that was created
updated_node = await tree_manager.get_node(current_node_id)
if updated_node.schema_linking:
    mapping = updated_node.schema_linking
    print(f"\n🗺️  Schema mapping created:")
    if mapping.get('tables'):
        print(f"   Tables: {[t['name'] for t in mapping['tables']]}")
    if mapping.get('columns'):
        print(f"   Columns: {len(mapping['columns'])} mapped")
        for col in mapping['columns'][:3]:  # Show first 3
            print(f"     - {col['table']}.{col['column']} ({col['usedFor']})")
        if len(mapping['columns']) > 3:
            print(f"     ... and {len(mapping['columns']) - 3} more")
else:
    print("⚠️  No schema mapping found")

2025-05-30 03:42:21,650 - SchemaLinkerAgent - INFO - Available tables in schema: ['frpm', 'satscores', 'schools']
2025-05-30 03:42:21,651 - SchemaLinkerAgent - INFO - Schema linking context prepared for node: root
2025-05-30 03:42:21,651 - SchemaLinkerAgent - INFO - Node details: {'nodeId': 'root', 'status': 'created', 'childIds': [], 'intent': 'What is the highest eligible free rate for K-12 students in the schools in Alameda County?', 'schema_linking': {}, 'generation': {}, 'evaluation': {}, 'evidence': 'Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`'}



🔗 STEP 1: SCHEMA LINKING
Working on node: root
Intent: What is the highest eligible free rate for K-12 students in the schools in Alameda County?


2025-05-30 03:42:38,355 - SchemaLinkerAgent - INFO - Raw LLM output: ```xml
<schema_linking>
  <available_schema>
    <tables>
      <table name="frpm">
        <columns>
          <column name="CDSCode" type="text" sample_values="['01', '02', '03']"/>
          <column name="County Name" type="text" sample_values="['Los Angeles', 'San Diego', 'Alameda']"/>
          <column name="Enrollment (K-12)" type="real" sample_values="[1000, 2000, 1500]"/>
          <column name="Free Meal Count (K-12)" type="real" sample_values="[500, 800, 600]"/>
          <column name="Percent (%) Eligible Free (K-12)" type="real" sample_values="[50.0, 40.0, 60.0]"/>
        </columns>
      </table>
      <table name="satscores">
        <columns>
          <column name="cds" type="text" sample_values="['01', '02', '03']"/>
          <column name="cname" type="text" sample_values="['Los Angeles', 'San Diego', 'Alameda']"/>
        </columns>
      </table>
      <table name="schools">
        <columns>
    

✅ Schema linking complete

🗺️  Schema mapping created:


In [8]:
# Step 2: Query Analysis - Analyze query and potentially decompose
print("\n🔍 STEP 2: QUERY ANALYSIS")
print("=" * 50)

# Run query analyzer to analyze the query (may decompose complex queries)
result2 = await query_analyzer.run(test_query)
print(f"✅ Query analysis complete")

# Check what was created or updated in the query tree
tree_stats = await tree_manager.get_tree_stats()
print(f"📊 Query tree after analysis:")
print(f"   - Total nodes: {tree_stats['total_nodes']}")
if tree_stats['total_nodes'] > 0:
    print(f"   - Max depth: {tree_stats.get('max_depth', 1)}")
    print(f"   - Leaf nodes: {tree_stats.get('leaf_count', 1)}")

    # Get root node to see if it was updated
    root_id = await tree_manager.get_root_id()
    root_node = await tree_manager.get_node(root_id)
    print(f"\n📄 Root node status:")
    print(f"   - ID: {root_id}")
    print(f"   - Intent: {root_node.intent}")
    print(f"   - Status: {root_node.status}")

    # Check if children were created (query decomposition)
    children = await tree_manager.get_children(root_id)
    if children:
        print(f"\n👥 Child nodes created (query decomposed): {len(children)}")
        for i, child in enumerate(children, 1):
            print(f"   {i}. {child.nodeId}: {child.intent}")
    else:
        print(f"\n👤 Single node query (no decomposition needed)")
else:
    print("⚠️  No query tree found - check analyzer output")

2025-05-30 03:42:38,361 - QueryAnalyzerAgent - INFO - Query analyzer context prepared with schema length: 9697
2025-05-30 03:42:38,362 - QueryAnalyzerAgent - INFO - query: What is the highest eligible free rate for K-12 students in the schools in Alameda County? database: california_schools
2025-05-30 03:42:38,362 - QueryAnalyzerAgent - INFO - evidence: Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`



🔍 STEP 2: QUERY ANALYSIS


2025-05-30 03:42:47,024 - QueryAnalyzerAgent - INFO - Raw LLM output: <analysis>
  <intent>Find the highest eligible free meal rate for K-12 students in schools located in Alameda County.</intent>
  <complexity>complex</complexity>
  <tables>
    <table name="frpm" purpose="Contains data on free meal counts and enrollment for K-12 students, necessary to calculate the eligible free rate."/>
    <table name="schools" purpose="Used to filter schools located in Alameda County."/>
  </tables>
  <decomposition>
    <subquery id="1">
      <intent>Calculate the eligible free rate for each school in Alameda County.</intent>
      <description>This subquery calculates the eligible free rate for K-12 students by dividing the 'Free Meal Count (K-12)' by 'Enrollment (K-12)' for each school in Alameda County. It filters the schools based on the 'County Name' being 'Alameda'.</description>
      <tables>frpm, schools</tables>
    </subquery>
    <subquery id="2">
      <intent>Find the highest eligi

✅ Query analysis complete
📊 Query tree after analysis:
   - Total nodes: 3
   - Max depth: 1
   - Leaf nodes: 1

📄 Root node status:
   - ID: root
   - Intent: Find the highest eligible free meal rate for K-12 students in schools located in Alameda County.
   - Status: NodeStatus.CREATED

👥 Child nodes created (query decomposed): 2
   1. node_1748590967.026294_1: Calculate the eligible free rate for each school in Alameda County.
   2. node_1748590967.027291_2: Find the highest eligible free rate from the calculated rates.


In [9]:
# Step 3: SQL Generation  
print("\n💻 STEP 3: SQL GENERATION")
print("=" * 50)

# Get current node to work with
current_node_id = await tree_manager.get_current_node_id()
print(f"Working on node: {current_node_id}")

# Run SQL generator
result3 = await sql_generator.run("Generate SQL query")
print(f"✅ SQL generation complete")

# Check the generated SQL
updated_node = await tree_manager.get_node(current_node_id)
if updated_node.generation and updated_node.generation.get('sql'):
    sql = updated_node.generation['sql']
    print(f"\n📝 Generated SQL:")
    print("─" * 40)
    print(sql)
    print("─" * 40)
    
    # Show any explanation if available
    if updated_node.generation.get('explanation'):
        explanation = updated_node.generation['explanation']
        print(f"\n💡 Explanation: {explanation[:200]}...")
else:
    print("⚠️  No SQL generated")

2025-05-30 03:42:47,034 - SQLGeneratorAgent - INFO - SQL generator context prepared for node: node_1748590967.026294_1
2025-05-30 03:42:47,034 - SQLGeneratorAgent - INFO - Node detail: {'nodeId': 'node_1748590967.026294_1', 'status': 'created', 'childIds': [], 'intent': 'Calculate the eligible free rate for each school in Alameda County.', 'schema_linking': {}, 'generation': {}, 'evaluation': {}, 'parentId': 'root'}



💻 STEP 3: SQL GENERATION
Working on node: node_1748590967.026294_1


2025-05-30 03:42:51,145 - SQLGeneratorAgent - INFO - Raw LLM output: <generation>
  <query_type>simple</query_type>
  <sql>
    SELECT 
      school_name, 
      (CAST(free_meal_count_k12 AS REAL) / CAST(enrollment_k12 AS REAL)) AS eligible_free_rate
    FROM 
      schools
    WHERE 
      county = 'Alameda'
  </sql>
  <explanation>
    The query calculates the eligible free rate for each school in Alameda County by dividing the `Free Meal Count (K-12)` by the `Enrollment (K-12)`. It selects the `school_name` and the calculated rate, filtering for schools located in Alameda County.
  </explanation>
  <considerations>
    - Assumes the table `schools` contains columns `school_name`, `free_meal_count_k12`, `enrollment_k12`, and `county`.
    - The division uses CAST to ensure floating-point division.
    - The WHERE clause filters schools specifically in Alameda County.
    - The calculation follows the formula provided in the evidence.
  </considerations>
</generation>
2025-05-30 03:42

✅ SQL generation complete

📝 Generated SQL:
────────────────────────────────────────
SELECT school_name, (CAST(free_meal_count_k12 AS REAL) / CAST(enrollment_k12 AS REAL)) AS eligible_free_rate FROM schools WHERE county = 'Alameda'
────────────────────────────────────────

💡 Explanation: The query calculates the eligible free rate for each school in Alameda County by dividing the `Free Meal Count (K-12)` by the `Enrollment (K-12)`. It selects the `school_name` and the calculated rate,...


In [10]:
# Step 4: SQL Execution & Evaluation
print("\n🚀 STEP 4: SQL EXECUTION & EVALUATION")
print("=" * 50)

# Run SQL evaluator (this will execute the SQL and evaluate results)
result4 = await sql_evaluator.run("Analyze SQL execution results")
print(f"✅ SQL execution and evaluation complete")

# Check execution results
current_node_id = await tree_manager.get_current_node_id()
updated_node = await tree_manager.get_node(current_node_id)
if updated_node.evaluation:
    evaluation = updated_node.evaluation
    print(f"\n📊 Execution Results:")
    
    # Show execution status
    if evaluation.get('execution_result'):
        exec_result = evaluation['execution_result']
        print(f"   Status: {exec_result.get('status', 'Unknown')}")
        print(f"   Rows returned: {exec_result.get('row_count', 0)}")
        
        if exec_result.get('data'):
            print(f"   Sample data: {exec_result['data'][:2]}...")  # First 2 rows
    
    # Show evaluation
    print(f"\n🎯 Quality Assessment:")
    print(f"   Answers intent: {evaluation.get('answers_intent', 'Unknown')}")
    print(f"   Result quality: {evaluation.get('result_quality', 'Unknown')}")
    print(f"   Confidence: {evaluation.get('confidence_score', 'N/A')}")
    
    if evaluation.get('result_summary'):
        summary = evaluation['result_summary']
        print(f"   Summary: {summary[:150]}...")
        
    # Show issues if any
    if evaluation.get('issues'):
        print(f"\n⚠️  Issues found: {evaluation['issues']}")
else:
    print("⚠️  No evaluation results found")

2025-05-30 03:42:51,153 - SQLEvaluatorAgent - INFO - Using current node: node_1748590967.026294_1
2025-05-30 03:42:51,154 - SQLEvaluatorAgent - WARNING - No data_path in database schema metadata, using default
2025-05-30 03:42:51,155 - QueryTreeManager - INFO - Updated node node_1748590967.026294_1
2025-05-30 03:42:51,155 - QueryTreeManager - INFO - Updated node node_1748590967.026294_1



🚀 STEP 4: SQL EXECUTION & EVALUATION
[SQLExecutor] Connecting to database: /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_databases/california_schools/california_schools.sqlite


2025-05-30 03:42:58,322 - SQLEvaluatorAgent - INFO - Raw LLM output: <evaluation>
  <answers_intent>no</answers_intent>
  <result_quality>poor</result_quality>
  <result_summary>The SQL query intended to calculate the eligible free rate for each school in Alameda County, but returned 0 rows, indicating that it failed to provide any results. This suggests an issue with the query logic or data availability.</result_summary>
  <issues>
    <issue>
      <type>completeness</type>
      <description>The query returned 0 rows, which is unexpected given the intent to calculate rates for schools in Alameda County.</description>
      <severity>high</severity>
    </issue>
    <issue>
      <type>logic</type>
      <description>The WHERE clause may be too restrictive or the data for Alameda County might be missing or incorrectly filtered.</description>
      <severity>high</severity>
    </issue>
  </issues>
  <suggestions>
    <suggestion>Verify that the 'schools' table contains data for Alame

✅ SQL execution and evaluation complete

📊 Execution Results:
   Status: success
   Rows returned: 0

🎯 Quality Assessment:
   Answers intent: no
   Result quality: poor
   Confidence: 0.9
   Summary: The SQL query intended to calculate the eligible free rate for each school in Alameda County, but returned 0 rows, indicating that it failed to provid...

⚠️  Issues found: {'issue': [{'type': 'completeness', 'description': 'The query returned 0 rows, which is unexpected given the intent to calculate rates for schools in Alameda County.', 'severity': 'high'}, {'type': 'logic', 'description': 'The WHERE clause may be too restrictive or the data for Alameda County might be missing or incorrectly filtered.', 'severity': 'high'}]}


In [11]:
# Step 5: Task Status Check
print("\n📍 STEP 5: TASK STATUS CHECK")
print("=" * 50)

# Check task status to see what should be done next
args = TaskStatusCheckerArgs(task="Check workflow completion status")
status_result = await task_status_checker.run(args)
print("TaskStatusChecker response:")
print(status_result)

# Parse the action
import re
action_match = re.search(r'ACTION:\s*(.+?)(?:\n|$)', status_result)
if action_match:
    action = action_match.group(1).strip()
    print(f"\n🎯 Recommended Action: {action}")
    
    # Extract node ID if present
    node_match = re.search(r'node:(\S+)', action) or re.search(r'NODE:\s*(\S+)', action)
    if node_match:
        recommended_node = node_match.group(1)
        print(f"📌 Target Node: {recommended_node}")
        
        # Set current node to the recommended one
        await tree_manager.set_current_node_id(recommended_node)
        print(f"✅ Current node updated to: {recommended_node}")
    
    # Check if workflow is complete
    if "TASK COMPLETE" in action.upper() or "COMPLETE" in action.upper():
        print("🏆 Workflow completed successfully!")
    elif "PROCESS NODE" in action.upper() or "RETRY" in action.upper():
        print("⚠️  More processing needed")
    else:
        print("❓ Action requires manual review")
else:
    print("⚠️  Could not parse action from TaskStatusChecker response")


📍 STEP 5: TASK STATUS CHECK
TaskStatusChecker response:
TREE OVERVIEW: 0/3 nodes complete
PENDING: 2 need SQL, 0 need eval, 1 bad SQL
CURRENT_NODE: node_1748590967.026294_1
CURRENT_STATUS: bad_sql
CURRENT_INTENT: Calculate the eligible free rate for each school in Alameda ...
OVERALL_STATUS: Processing in progress
⚠️  Could not parse action from TaskStatusChecker response


In [12]:
async def display_memory_summary():
    """Display a summary of memory contents"""
    print("🧠 MEMORY CONTENTS SUMMARY")
    print("=" * 60)
    
    # Get all memory data
    memory_data = await memory.show_all(format="json")
    
    if not memory_data:
        print("Memory is empty")
        return
    
    print(f"📦 Total memory keys: {len(memory_data)}")
    
    for key, value in memory_data.items():
        print(f"\n📁 {key}")
        
        if key == "queryTree":
            tree = value["value"]
            nodes = tree.get("nodes", {})
            print(f"   - Root ID: {tree.get('rootId')}")
            print(f"   - Current Node: {tree.get('currentNodeId')}")
            print(f"   - Total Nodes: {len(nodes)}")
            
        elif key == "taskContext":
            task = value["value"]
            print(f"   - Task ID: {task.get('taskId')}")
            print(f"   - Query: {task.get('originalQuery', '')[:50]}...")
            print(f"   - Database: {task.get('databaseName')}")
            print(f"   - Status: {task.get('status')}")
            
        elif key == "databaseSchema":
            schema = value["value"]
            tables = schema.get("tables", {})
            print(f"   - Tables: {len(tables)}")
            print(f"   - Table names: {list(tables.keys())}")
            
        elif key == "nodeHistory":
            history = value["value"]
            print(f"   - Total operations: {len(history)}")
            print(f"   - Operation types: {list(set(op.get('operation') for op in history))}")
            
        elif key.endswith("_analysis"):
            analysis = value["value"]
            print(f"   - Quality: {analysis.get('result_quality', 'N/A')}")
            print(f"   - Answers intent: {analysis.get('answers_intent', 'N/A')}")
            
        else:
            # Show first few characters for other keys
            content = str(value["value"])
            preview = content[:100] + "..." if len(content) > 100 else content
            print(f"   - Content: {preview}")

# Display memory summary
await display_memory_summary()

🧠 MEMORY CONTENTS SUMMARY
📦 Total memory keys: 4

📁 queryTree
   - Root ID: root
   - Current Node: node_1748590967.026294_1
   - Total Nodes: 3

📁 nodeHistory
   - Total operations: 5
   - Operation types: ['create', 'generate_sql']

📁 databaseSchema
   - Tables: 3
   - Table names: ['frpm', 'satscores', 'schools']

📁 taskContext
   - Task ID: workflow_demo_001
   - Query: What is the highest eligible free rate for K-12 st...
   - Database: california_schools
   - Status: initializing
